In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
from groclient import GroClient
import os

In [2]:
# setup client
API_HOST = 'api.gro-intelligence.com'
GROAPI_TOKEN = os.environ['GROAPI_TOKEN']
client = GroClient(API_HOST, GROAPI_TOKEN)

to do:
- write a function with region id as input, output balance sheet
    - item/metrics different for soybeans and corn
- can write different functions for different crops now
- start with soybeans
- for yield: have one row that is PSD yield, one row that is Gro yield (when available, figure out date alignment)

for date alignment:

e.g. current year is Sept 2021 - Aug 2022 for US. Corresponds with Feb 22-Jan 23 Brazil and March 22-Feb 23 Argentina.

SUPPLY:
* beginning carryover - PS&D
* (yield)
* harvested area - PS&D
* production - PS&D
* imports - PS&D
* total supply - PS&D

DEMAND:
* crush - PS&D
* food use - PS&D
* waste and feed - PS&D
* domestic consumption - PS&D
* exports - PS&D
* total demand

STOCKS:
* ending carryover - PS&D
* carryover, weeks of total use
* ending stocks to use

In [8]:
metric_dict = {
    1450032: {'group': 'Supply'}, # beginning carryover/stocks, beginning quantity
    570001: {'group': 'Supply'}, # harvested area
    860032: {'group': 'Supply'}, # production
    30032: {'group': 'Supply'}, # imports
    170037: {'group': 'Supply'}, # yield
    1880032: {'group': 'Supply'}, # total supply
    1800032: {'group': 'Demand'}, # crush
    1930000: {'group': 'Demand'}, # food use
    1760032: {'group': 'Demand'}, # feed use
    1480032: {'group': 'Demand'}, # domestic consumption
    20032: {'group': 'Demand'}, # exports
    1470032: {'group': 'Stocks'} # ending carryover/stocks, ending quantity
}

for entry in metric_dict.keys():
    metric_name = client.lookup('metrics', entry)['name']
    if 'Area' not in metric_name and 'Yield' not in metric_name:
        unit_id = 14 # tonnes
    elif 'Yield' in metric_name:
        unit_id = 61 # tonne / hectare
        metric_name = 'Yield (mass / area) (Gro Yield Model)'
    else:
        unit_id = 42 # hectares
    metric_dict[entry]['name'] = metric_name
    metric_dict[entry]['unit_id'] = unit_id
    
metric_dict

{1450032: {'group': 'Supply',
  'name': 'Stocks, Beginning Quantity (mass)',
  'unit_id': 14},
 570001: {'group': 'Supply', 'name': 'Area Harvested', 'unit_id': 42},
 860032: {'group': 'Supply',
  'name': 'Production Quantity (mass)',
  'unit_id': 14},
 30032: {'group': 'Supply', 'name': 'Import Volume (mass)', 'unit_id': 14},
 170037: {'group': 'Supply',
  'name': 'Yield (mass / area) (Gro Yield Model)',
  'unit_id': 61},
 1880032: {'group': 'Supply', 'name': 'Supply Quantity (mass)', 'unit_id': 14},
 1800032: {'group': 'Demand', 'name': 'Crushing (mass)', 'unit_id': 14},
 1930000: {'group': 'Demand', 'name': 'Food Use', 'unit_id': 14},
 1760032: {'group': 'Demand', 'name': 'Feed Use', 'unit_id': 14},
 1480032: {'group': 'Demand',
  'name': 'Domestic Consumption (mass)',
  'unit_id': 14},
 20032: {'group': 'Demand', 'name': 'Export Volume (mass)', 'unit_id': 14},
 1470032: {'group': 'Stocks',
  'name': 'Stocks, Ending Quantity (mass)',
  'unit_id': 14}}

In [32]:
def get_soybeans_balance_sheet(client, regions, metric_dict):
    braz_arg_regions = [region for region in regions if region == 1029 or region == 1010]
    non_braz_arg_regions = [region for region in regions if region != 1029 and region != 1010]
    
    base_query1 = {
        'item_id': 270, # soybeans
        'region_id': non_braz_arg_regions,
        'source_id': 14, # USDA PS&D
        'frequency_id': 9 # annual
    }
    
    base_query2 = {
        'item_id': 12418, # soybeans, local
        'region_id': braz_arg_regions,
        'source_id': 14, # USDA PS&D
        'frequency_id': 9 # annual
    }
    just_us = (regions == [1215]) # do unit conversion if the input is just the US
    
    # query for all mass metrics from PS&D
    mass_query1 = base_query1.copy()
    mass_query1['metric_id'] = [key for key in metric_dict.keys() if metric_dict[key]['unit_id'] == 14]
    mass_query1['unit_id'] = 20 if just_us else 14
    
    mass_query2 = base_query2.copy()
    mass_query2['metric_id'] = [key for key in metric_dict.keys() if metric_dict[key]['unit_id'] == 14]
    mass_query2['unit_id'] = 20 if just_us else 14
    
    # query for planted area
    area_query1 = base_query1.copy()
    area_query1['metric_id'] = [key for key in metric_dict.keys() if metric_dict[key]['unit_id'] == 42]
    area_query1['unit_id'] = 41 if just_us else 42
    
    area_query2 = base_query2.copy()
    area_query2['metric_id'] = [key for key in metric_dict.keys() if metric_dict[key]['unit_id'] == 42]
    area_query2['unit_id'] = 41 if just_us else 42
    
    # query for Gro yield
    yield_query = {
        'metric_id': 170037, # yield (mass/area)
        'item_id': 270, # soybeans
        'region_id': regions,
        'source_id': 32, # Gro Yield Model
        'frequency_id': 9, # annual
        'unit_id': 287 if just_us else 61 # tonne per hectare unless just US in which case bu/acre
    }
    
    mass_area_dfs = []
    if len(non_braz_arg_regions) != 0:
        mass_area_dfs.append(pd.DataFrame(client.get_data_points(**mass_query1)))
        mass_area_dfs.append(pd.DataFrame(client.get_data_points(**area_query1)))
    if len(braz_arg_regions) != 0:
        mass_area_dfs.append(pd.DataFrame(client.get_data_points(**mass_query2)))
        mass_area_dfs.append(pd.DataFrame(client.get_data_points(**area_query2)))
    
    gro_yield = pd.DataFrame(client.get_data_points(**yield_query))
    
    # combine to get all metrics together
    all_metrics = pd.concat(mass_area_dfs, axis=0)
    all_metrics['group'] = all_metrics['metric_id'].apply(lambda x: metric_dict[x]['group'])
    all_metrics['metric_name'] = all_metrics['metric_id'].apply(lambda x: metric_dict[x]['name'])
    all_metrics['northern_hemisphere'] = all_metrics['region_id'].apply(lambda x: client.lookup('regions', 
                                                                                                x)['latitude'] > 0)
    gro_yield['northern_hemisphere'] = gro_yield['region_id'].apply(lambda x: client.lookup('regions', 
                                                                                              x)['latitude'] > 0)
    
    def get_year(n_hemisphere, end_date, start_date):
        return pd.to_datetime(end_date).year if n_hemisphere else pd.to_datetime(start_date).year
            
    all_metrics['year'] = (all_metrics[['northern_hemisphere', 'end_date', 'start_date']]
                           .apply(lambda x: get_year(x[0], x[1], x[2]), axis=1))
    gro_yield['year'] = (gro_yield[['northern_hemisphere', 'end_date']]
                         .apply(lambda x: pd.to_datetime(x[1]).year + 1 if x[0] 
                                else pd.to_datetime(x[1]).year, axis=1))
    
    all_metrics_ = all_metrics[['year', 'region_id', 'value', 'metric_name', 'group']]
    
    max_start_year = max(all_metrics_.groupby('region_id')['year'].min())
    min_end_year = min(all_metrics_.groupby('region_id')['year'].max())
    
    # separate into supply, demand, and stocks
    supply = all_metrics_[all_metrics_['group'] == 'Supply'].drop('group', axis=1)
    demand = all_metrics_[all_metrics_['group'] == 'Demand'].drop('group', axis=1)
    stocks = all_metrics_[all_metrics_['group'] == 'Stocks'].drop('group', axis=1)
    
    # get in proper format with years as columns
    # add all relevant countries together
    all_outputs = []
    
    total_demand_col = None
    for i, df in enumerate([supply, demand, stocks]):
        df_pivot = df.pivot(index=['year', 'region_id'], columns='metric_name', values='value').reset_index()
        df_out = (df_pivot.query('region_id == @regions[0]').drop('region_id', axis=1)
                  .pivot_table(columns='year').loc[:, max_start_year:min_end_year])
        for region in regions[1:]:
            df_out = df_out.add(df_pivot.query('region_id == @region').drop('region_id', axis=1)
                                .pivot_table(columns='year').loc[:, max_start_year:min_end_year])
        if i == 0: # supply
            df_out.loc['Yield (USDA PS&D)', :] = (df_out.loc['Production Quantity (mass)', :]
                                           .divide(df_out.loc['Area Harvested', :]))
            yield_start_year = max(gro_yield.groupby('region_id')['year'].min())
            gro_yield_pivot = (gro_yield.groupby(['year'])['value'].mean().reset_index()
                               .pivot_table(columns=['year']).loc[:, yield_start_year:min_end_year])
            df_out.loc['Yield (mass / area) (Gro Yield Model)', 
                       yield_start_year:min_end_year] = gro_yield_pivot.values 
            df_out.loc['Yield (mass / area) (Gro Yield Model)', 
                       max_start_year:yield_start_year] = np.nan
            
        if i == 1: # demand
            df_out.loc['Total Demand (mass)', :] = (df_out.loc['Domestic Consumption (mass)', :]
                                                    .add(df_out.loc['Export Volume (mass)', :]))
            total_demand_col = df_out.loc['Total Demand (mass)', :]
        
        if i == 2: # stocks
            df_out.loc['Carryover, Weeks of Total Use (mass)', :] = (df_out.loc['Stocks, Ending Quantity (mass)']
                                                                     .divide(total_demand_col / 52))
            df_out.loc['Ending Stocks-to-Use', :] = (df_out.loc['Stocks, Ending Quantity (mass)']
                                                     .divide(total_demand_col))
            
        all_outputs.append(df_out)
        
    return all_outputs

In [33]:
supply, demand, stocks = get_soybeans_balance_sheet(client, [1215, 1029, 1010, 1231], metric_dict)

display(supply)
display(demand)
display(stocks)

supply.to_csv('output_supply.csv')
demand.to_csv('output_demand.csv')
stocks.to_csv('output_stocks.csv')

year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
Area Harvested,2.291900e+07,2.304100e+07,2.384400e+07,2.535400e+07,2.604800e+07,2.640300e+07,2.683500e+07,2.798100e+07,2.984300e+07,3.542300e+07,...,8.566900e+07,8.725000e+07,9.197900e+07,9.255700e+07,9.230400e+07,9.593100e+07,9.636100e+07,9.325900e+07,9.868100e+07,9.993700e+07
Import Volume (mass),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+03,7.000000e+03,2.600000e+05,6.250000e+05,...,6.121000e+07,7.289800e+07,7.972400e+07,8.553700e+07,9.699900e+07,1.021350e+08,8.685700e+07,1.047760e+08,1.059520e+08,9.776400e+07
Production Quantity (mass),2.748600e+07,2.976700e+07,3.427600e+07,3.552100e+07,3.925600e+07,4.000500e+07,4.152100e+07,4.436300e+07,4.631500e+07,5.886000e+07,...,2.275270e+08,2.433700e+08,2.781410e+08,2.737360e+08,3.004270e+08,2.965480e+08,3.114820e+08,2.920590e+08,3.185510e+08,3.076070e+08
"Stocks, Beginning Quantity (mass)",1.888000e+06,9.910000e+05,1.182000e+06,2.595000e+06,4.706000e+06,9.093000e+06,6.573000e+06,2.893000e+06,2.333000e+06,2.042000e+06,...,2.880900e+07,2.626700e+07,2.920400e+07,3.434700e+07,3.725800e+07,4.808300e+07,4.672400e+07,5.694600e+07,5.446500e+07,5.195700e+07
Supply Quantity (mass),2.937400e+07,3.075800e+07,3.545800e+07,3.811600e+07,4.396200e+07,4.909800e+07,4.809500e+07,4.726300e+07,4.890800e+07,6.152700e+07,...,3.175460e+08,3.425350e+08,3.870690e+08,3.936200e+08,4.346840e+08,4.467660e+08,4.450630e+08,4.537810e+08,4.789680e+08,4.573280e+08
Yield (USDA PS&D),1.199267e+00,1.291914e+00,1.437510e+00,1.401002e+00,1.507064e+00,1.515169e+00,1.547270e+00,1.585469e+00,1.551955e+00,1.661632e+00,...,2.655885e+00,2.789341e+00,3.023962e+00,2.957486e+00,3.254756e+00,3.091264e+00,3.232449e+00,3.131698e+00,3.228088e+00,3.078009e+00
Yield (mass / area) (Gro Yield Model),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.720302e+00,2.985405e+00,3.145719e+00,3.186169e+00,3.125198e+00,2.964115e+00,3.305458e+00,3.086726e+00,3.237788e+00,3.103889e+00


year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
Crushing (mass),14628000.0,15994000.0,17041000.0,17558000.0,18477000.0,22316000.0,23933000.0,23300000.0,23699000.0,28535000.0,...,182408000.0,192790000.0,210924000.0,215844000.0,222843000.0,225674000.0,226618000.0,234710000.0,240157000.0,237982000.0
Domestic Consumption (mass),21957000.0,22085000.0,24874000.0,25518000.0,26266000.0,30038000.0,32706000.0,32193000.0,31720000.0,38298000.0,...,204241000.0,215341000.0,236138000.0,242234000.0,251470000.0,254268000.0,256589000.0,265239000.0,271519000.0,270698000.0
Export Volume (mass),6426000.0,7491000.0,7989000.0,7892000.0,8603000.0,12487000.0,12496000.0,12737000.0,15146000.0,17875000.0,...,87038000.0,97990000.0,116584000.0,114128000.0,135131000.0,145774000.0,131528000.0,134077000.0,155492000.0,138480000.0
Feed Use,2113000.0,2225000.0,2281000.0,2403000.0,2347000.0,2532000.0,2734000.0,2845000.0,3439000.0,3366000.0,...,12133000.0,12551000.0,14614000.0,15190000.0,16727000.0,16194000.0,17071000.0,17129000.0,17362000.0,17916000.0
Total Demand (mass),28383000.0,29576000.0,32863000.0,33410000.0,34869000.0,42525000.0,45202000.0,44930000.0,46866000.0,56173000.0,...,291279000.0,313331000.0,352722000.0,356362000.0,386601000.0,400042000.0,388117000.0,399316000.0,427011000.0,409178000.0


year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
"Stocks, Ending Quantity (mass)",991000.000000,1.182000e+06,2.595000e+06,4.706000e+06,9.093000e+06,6.573000e+06,2.893000e+06,2.333000e+06,2.042000e+06,5.354000e+06,...,2.626700e+07,2.920400e+07,3.434700e+07,3.725800e+07,4.808300e+07,4.672400e+07,5.694600e+07,5.446500e+07,5.195700e+07,4.815000e+07
"Carryover, Weeks of Total Use (mass)",1.815594,2.078171e+00,4.106138e+00,7.324514e+00,1.356035e+01,8.037531e+00,3.328083e+00,2.700111e+00,2.265694e+00,4.956260e+00,...,4.689264e+00,4.846657e+00,5.063602e+00,5.436651e+00,6.467433e+00,6.073482e+00,7.629637e+00,7.092578e+00,6.327153e+00,6.119097e+00
Ending Stocks-to-Use,0.034915,3.996484e-02,7.896418e-02,1.408560e-01,2.607760e-01,1.545679e-01,6.400159e-02,5.192522e-02,4.357103e-02,9.531269e-02,...,9.017815e-02,9.320495e-02,9.737697e-02,1.045510e-01,1.243737e-01,1.167977e-01,1.467238e-01,1.363957e-01,1.216760e-01,1.176749e-01


In [34]:
supply, demand, stocks = get_soybeans_balance_sheet(client, [1215], metric_dict)

display(supply)
display(demand)
display(stocks)

year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
Area Harvested,3.079427e+07,3.444896e+07,3.654689e+07,3.980621e+07,4.139262e+07,4.133826e+07,4.225008e+07,4.270475e+07,4.568484e+07,5.566790e+07,...,7.614305e+07,7.623201e+07,8.260980e+07,8.174246e+07,8.270617e+07,8.954111e+07,8.759392e+07,7.493965e+07,8.260239e+07,8.633121e+07
Import Volume (mass),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.052863e+07,7.176103e+07,3.321658e+07,2.355290e+07,2.226686e+07,2.182594e+07,1.407295e+07,1.539574e+07,1.984176e+07,1.499155e+07
Production Quantity (mass),7.009285e+08,8.456264e+08,9.284841e+08,9.764718e+08,1.106986e+09,1.133148e+09,1.127122e+09,1.176139e+09,1.270644e+09,1.547584e+09,...,3.042073e+09,3.357042e+09,3.928117e+09,3.926795e+09,4.296513e+09,4.411668e+09,4.428203e+09,3.551932e+09,4.216337e+09,4.435258e+09
"Stocks, Beginning Quantity (mass)",6.731501e+07,2.968915e+07,3.564168e+07,9.013303e+07,1.663401e+08,3.268379e+08,2.298337e+08,9.876787e+07,7.194475e+07,5.963551e+07,...,1.693898e+08,1.405458e+08,9.200698e+07,1.906279e+08,1.967274e+08,3.015948e+08,4.380987e+08,9.090466e+08,5.245573e+08,2.569875e+08
Supply Quantity (mass),7.682436e+08,8.753156e+08,9.641258e+08,1.066605e+09,1.273327e+09,1.459986e+09,1.356956e+09,1.274907e+09,1.342589e+09,1.607219e+09,...,3.251991e+09,3.569349e+09,4.053341e+09,4.140975e+09,4.515507e+09,4.735089e+09,4.880375e+09,4.476375e+09,4.760736e+09,4.707237e+09
Yield (USDA PS&D),2.276165e+01,2.454723e+01,2.540529e+01,2.453064e+01,2.674357e+01,2.741161e+01,2.667740e+01,2.754117e+01,2.781326e+01,2.780029e+01,...,3.995207e+01,4.403717e+01,4.755026e+01,4.803861e+01,5.194912e+01,4.926976e+01,5.055378e+01,4.739724e+01,5.104377e+01,5.137491e+01
Yield (mass / area) (Gro Yield Model),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.727782e+01,4.458152e+01,4.772957e+01,4.762359e+01,4.980140e+01,4.971373e+01,5.061013e+01,4.664752e+01,5.144973e+01,5.111652e+01


year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
Crushing (mass),4.789948e+08,5.374912e+08,5.594274e+08,5.764031e+08,6.059086e+08,7.373051e+08,7.601231e+08,7.205498e+08,7.217991e+08,8.213386e+08,...,1.689011e+09,1.734023e+09,1.873025e+09,1.886253e+09,1.901208e+09,2.054945e+09,2.092020e+09,2.164589e+09,2.140595e+09,2.215002e+09
Domestic Consumption (mass),5.263945e+08,5.890798e+08,6.124122e+08,6.336870e+08,6.597018e+08,7.975653e+08,8.243884e+08,7.861379e+08,8.035178e+08,8.973252e+08,...,1.783921e+09,1.838780e+09,2.020516e+09,2.001629e+09,2.047339e+09,2.163230e+09,2.217868e+09,2.272580e+09,2.243184e+09,2.331884e+09
Export Volume (mass),2.121599e+08,2.505941e+08,2.615805e+08,2.665777e+08,2.867869e+08,4.325871e+08,4.337997e+08,4.168239e+08,4.794357e+08,5.391447e+08,...,1.327524e+09,1.638562e+09,1.842197e+09,1.942619e+09,2.166573e+09,2.133761e+09,1.753460e+09,1.679238e+09,2.260564e+09,2.089999e+09
Feed Use,4.739976e+07,5.158858e+07,5.298485e+07,5.728390e+07,5.379322e+07,6.026016e+07,6.426526e+07,6.558804e+07,8.171866e+07,7.598659e+07,...,9.490975e+07,1.047571e+08,1.474904e+08,1.153762e+08,1.461309e+08,1.082846e+08,1.258482e+08,1.079906e+08,1.025892e+08,1.168827e+08
Total Demand (mass),7.385544e+08,8.396739e+08,8.739928e+08,9.002647e+08,9.464887e+08,1.230152e+09,1.258188e+09,1.202962e+09,1.282954e+09,1.436470e+09,...,3.111445e+09,3.477342e+09,3.862713e+09,3.944248e+09,4.213912e+09,4.296990e+09,3.971328e+09,3.951817e+09,4.503749e+09,4.421883e+09


year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
"Stocks, Ending Quantity (mass)",2.968915e+07,3.564168e+07,9.013303e+07,1.663401e+08,3.268379e+08,2.298337e+08,9.876787e+07,7.194475e+07,5.963551e+07,1.707494e+08,...,1.405458e+08,9.200698e+07,1.906279e+08,1.967274e+08,3.015948e+08,4.380987e+08,9.090466e+08,5.245573e+08,2.569875e+08,2.853539e+08
"Carryover, Weeks of Total Use (mass)",2.090348e+00,2.207247e+00,5.362650e+00,9.607934e+00,1.795644e+01,9.715344e+00,4.082005e+00,3.109930e+00,2.417115e+00,6.181102e+00,...,2.348870e+00,1.375868e+00,2.566240e+00,2.593606e+00,3.721702e+00,5.301649e+00,1.190292e+01,6.902390e+00,2.967162e+00,3.355675e+00
Ending Stocks-to-Use,4.019900e-02,4.244705e-02,1.031279e-01,1.847680e-01,3.453162e-01,1.868335e-01,7.850009e-02,5.980635e-02,4.648299e-02,1.188673e-01,...,4.517059e-02,2.645900e-02,4.935077e-02,4.987703e-02,7.157120e-02,1.019548e-01,2.289024e-01,1.327383e-01,5.706081e-02,6.453221e-02


In [35]:
supply, demand, stocks = get_soybeans_balance_sheet(client, [1029, 1010], metric_dict)

display(supply)
display(demand)
display(stocks)

year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
Area Harvested,448000.000000,507000.000000,629000.000000,742000.000000,9.340000e+05,1.345000e+06,1.752000e+06,2.908000e+06,3.772000e+06,5.487000e+06,...,4.745000e+07,4.935000e+07,5.145000e+07,5.265000e+07,5.123500e+07,5.145000e+07,5.250000e+07,5.360000e+07,5.537000e+07,5.660000e+07
Import Volume (mass),0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e+03,5.000000e+03,5.000000e+03,6.000000e+03,...,2.420000e+05,5.810000e+05,4.700000e+05,1.666000e+06,2.898000e+06,7.446000e+06,3.934000e+06,5.824000e+06,5.650000e+06,3.356000e+06
Production Quantity (mass),540000.000000,613000.000000,737000.000000,676000.000000,1.089000e+06,1.536000e+06,2.136000e+06,3.744000e+06,5.284000e+06,8.372000e+06,...,1.313000e+08,1.396000e+08,1.585500e+08,1.545000e+08,1.699000e+08,1.612000e+08,1.750000e+08,1.773000e+08,1.842000e+08,1.705000e+08
"Stocks, Beginning Quantity (mass)",56000.000000,183000.000000,212000.000000,142000.000000,1.790000e+05,1.980000e+05,3.180000e+05,2.050000e+05,3.750000e+05,4.190000e+05,...,8.123000e+06,1.003100e+07,1.273300e+07,1.209900e+07,1.526100e+07,1.975500e+07,1.173700e+07,1.275100e+07,1.339900e+07,1.047900e+07
Supply Quantity (mass),596000.000000,796000.000000,949000.000000,818000.000000,1.268000e+06,1.734000e+06,2.455000e+06,3.954000e+06,5.664000e+06,8.797000e+06,...,1.396650e+08,1.502120e+08,1.717530e+08,1.682650e+08,1.880590e+08,1.884010e+08,1.906710e+08,1.958750e+08,2.032490e+08,1.843350e+08
Yield (USDA PS&D),1.205357,1.209073,1.171701,0.911051,1.165953e+00,1.142007e+00,1.219178e+00,1.287483e+00,1.400848e+00,1.525788e+00,...,2.767123e+00,2.828774e+00,3.081633e+00,2.934473e+00,3.316093e+00,3.133139e+00,3.333333e+00,3.307836e+00,3.326711e+00,3.012367e+00
Yield (mass / area) (Gro Yield Model),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.826976e+00,2.979042e+00,3.113659e+00,3.177897e+00,3.013212e+00,2.774534e+00,3.207358e+00,3.061554e+00,3.126671e+00,2.937027e+00


year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
Crushing (mass),288000.0,400000.0,428000.0,482000.0,627000.0,953000.0,1736000.0,2178000.0,2909000.0,4583000.0,...,71441000.0,76698000.0,85449000.0,83009000.0,83101000.0,79748000.0,84683000.0,84300000.0,88900000.0,85700000.0
Domestic Consumption (mass),338000.0,463000.0,502000.0,573000.0,760000.0,1126000.0,2020000.0,2556000.0,3457000.0,5228000.0,...,78991000.0,84298000.0,93349000.0,91859000.0,92251000.0,89095000.0,94229000.0,94190000.0,98870000.0,95535000.0
Export Volume (mass),75000.0,121000.0,305000.0,66000.0,310000.0,290000.0,230000.0,1023000.0,1788000.0,2862000.0,...,50643000.0,53181000.0,66305000.0,61145000.0,76053000.0,87569000.0,83691000.0,88286000.0,93900000.0,81500000.0
Feed Use,50000.0,63000.0,74000.0,91000.0,133000.0,173000.0,284000.0,378000.0,548000.0,645000.0,...,7550000.0,7600000.0,7900000.0,8850000.0,9150000.0,9347000.0,9546000.0,9890000.0,9970000.0,9835000.0
Total Demand (mass),413000.0,584000.0,807000.0,639000.0,1070000.0,1416000.0,2250000.0,3579000.0,5245000.0,8090000.0,...,129634000.0,137479000.0,159654000.0,153004000.0,168304000.0,176664000.0,177920000.0,182476000.0,192770000.0,177035000.0


year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
metric_name,,,,,,,,,,,,,,,,,,,,,
"Stocks, Ending Quantity (mass)",183000.000000,212000.000000,142000.000000,179000.000000,198000.000000,318000.000000,205000.000000,375000.000000,419000.000000,707000.000000,...,1.003100e+07,1.273300e+07,1.209900e+07,1.526100e+07,1.975500e+07,1.173700e+07,1.275100e+07,1.339900e+07,1.047900e+07,7.300000e+06
"Carryover, Weeks of Total Use (mass)",23.041162,18.876712,9.149938,14.566510,9.622430,11.677966,4.737778,5.448449,4.154051,4.544376,...,4.023728e+00,4.816125e+00,3.940697e+00,5.186610e+00,6.103598e+00,3.454716e+00,3.726686e+00,3.818299e+00,2.826726e+00,2.144209e+00
Ending Stocks-to-Use,0.443099,0.363014,0.175960,0.280125,0.185047,0.224576,0.091111,0.104778,0.079886,0.087392,...,7.737939e-02,9.261778e-02,7.578263e-02,9.974249e-02,1.173769e-01,6.643685e-02,7.166704e-02,7.342883e-02,5.436012e-02,4.123478e-02
